In [8]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
tfe = tf.contrib.eager
try:
    tfe.enable_eager_execution()
except ValueError:
    pass

import matplotlib.pyplot as plt

from __future__ import print_function

In [9]:
# copypasta from https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Bayesian_Gaussian_Mixture_Model.ipynb

In [10]:
def session_options(enable_gpu_ram_resizing=True):
  """Convenience function which sets common `tf.Session` options."""
  config = tf.ConfigProto()
  config.log_device_placement = True
  if enable_gpu_ram_resizing:
    # `allow_growth=True` makes it possible to connect multiple colabs to your
    # GPU. Otherwise the colab malloc's all GPU ram.
    config.gpu_options.allow_growth = True
  return config

def reset_sess(config=None):
  """Convenience function to create the TF graph and session, or reset them."""
  if config is None:
    config = session_options()
  tf.reset_default_graph()
  global sess
  try:
    sess.close()
  except:
    pass
  sess = tf.InteractiveSession(config=config)

reset_sess()

In [11]:
#class MVNCholPrecisionTriL(tfd.TransformedDistribution):
#  """MVN from loc and (Cholesky) precision matrix."""
#
#  def __init__(self, loc, chol_precision_tril, name=None):
#    super(MVNCholPrecisionTriL, self).__init__(
#        distribution=tfd.Independent(tfd.Normal(tf.zeros_like(loc),
#                                                scale=tf.ones_like(loc)),
#                                     reinterpreted_batch_ndims=1),
#        bijector=tfb.Chain([
#            tfb.Affine(shift=loc),
#            tfb.Invert(tfb.Affine(scale_tril=chol_precision_tril,
#                                  adjoint=True)),
#        ]),
#        name=name)

In [11]:
dtype = np.float32
#dims = 2
dims = 1
components = 3

In [27]:
rv_mix_probs = tfp.edward2.Dirichlet(
    concentration=np.ones(components, dtype) / 10.,
    name='rv_mix_probs')

rv_loc = tfp.edward2.Independent(
    tfp.edward2.Normal(
        loc=np.stack([
            -np.ones(dims, dtype),
            np.zeros(dims, dtype),
            np.ones(dims, dtype),
        ]),
        scale=tf.ones([components, dims], dtype)),
    reinterpreted_batch_ndims=1,
    name='rv_loc')

#rv_precision = tfp.edward2.Wishart(
#    df=5,
#    scale_tril=np.stack([np.eye(dims, dtype=dtype)]*components),
#    input_output_cholesky=True,
#    name='rv_precision')
rv_precision = tfp.edward2.Gamma(
    concentration=1.,
    rate=1.
)

AttributeError: 'RandomVariable' object has no attribute 'reparameterization_type'

In [13]:
print(rv_mix_probs)
print(rv_loc)
print(rv_precision)

tf.distributions.Dirichlet("rv_mix_probs/", batch_shape=(), event_shape=(3,), dtype=float32)
tf.distributions.Independent("rv_loc/", batch_shape=(3,), event_shape=(1,), dtype=float32)
tf.distributions.Gamma("Gamma/", batch_shape=(), event_shape=(), dtype=float32)


In [20]:
rv_mix_probs

<tf.distributions.Dirichlet 'rv_mix_probs/' batch_shape=() event_shape=(3,) dtype=float32>

In [25]:
rv_mix_probs2 = tfp.edward2.Dirichlet(concentration=np.ones(components, dtype) / 10.,
    name='rv_mix_probs')

In [26]:
tfp.edward2.Categorical(probs=rv_mix_probs2)

<ed.RandomVariable 'Categorical_3/' shape=() dtype=int32>

In [27]:
rv_mix_probs.sample()

<tf.Tensor: id=62, shape=(3,), dtype=float32, numpy=array([4.8431428e-03, 4.1213221e-05, 9.9511564e-01], dtype=float32)>

In [24]:
tf.reduce_sum(rv_mix_probs.sample()).eval()

1.0

In [17]:
rv_observations = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(probs=rv_mix_probs),
    #components_distribution=MVNCholPrecisionTriL(
    #    loc=loc,
    #    chol_precision_tril=chol_precision))
    components_distribution=tfd.Normal(loc=rv_loc,scale=rv_precision))
  #log_prob_parts = [
  #    rv_observations.log_prob(observations), # Sum over samples.
  #    rv_mix_probs.log_prob(mix_probs)[..., tf.newaxis],
  #    rv_loc.log_prob(loc),                   # Sum over components.
  #    rv_precision.log_prob(chol_precision),  # Sum over components.
  #]
  #sum_log_prob = tf.reduce_sum(tf.concat(log_prob_parts, axis=-1), axis=-1)
  # Note: for easy debugging, uncomment the following:
  # sum_log_prob = tf.Print(sum_log_prob, log_prob_parts)

ValueError: Attempt to convert a value (<tf.distributions.Dirichlet 'rv_mix_probs/' batch_shape=() event_shape=(3,) dtype=float32>) with an unsupported type (<class 'tensorflow.python.ops.distributions.dirichlet.Dirichlet'>) to a Tensor.